In [ ]:
from __future__ import annotations

import importlib
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

sys.path.append(str(Path("__file__").resolve().parent.parent.parent))

from sub import estimate

importlib.reload(estimate)

plt.rcParams["figure.figsize"] = (16.0, 6.0)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

LOG_FILE_PATH = "../../../dataset/sample_trials/4_1_51.txt"
GIS_BASE_PATH = "../../../dataset/gis/"
BEACON_LIST_PATH = GIS_BASE_PATH + "beacon_list.csv"
FLOOR_NAMES = ["FLU01", "FLU02", "FLD01"]
fig_size = (10, 10)
# 1 pixel of bmp represents 0.01 m
dx = 0.01
dy = 0.01

In [ ]:
from typing import Literal

from sub import pdr

Axis2D = Literal["x", "y"]
importlib.reload(pdr)

data = pdr.read_log_data(LOG_FILE_PATH)
acc_df, gyro_df, mgf_df, ground_truth_df, ble_scans_df = pdr.convert_to_dataframes(data)
print(ble_scans_df)

ble_scans_df.to_csv("ble_scans.csv", index=False)


ble_positions_df = pd.read_csv(BEACON_LIST_PATH)
floor_name = ground_truth_df["floor_name"][0]
map_dict = pdr.load_floor_maps(FLOOR_NAMES, GIS_BASE_PATH)

true_point: dict[Axis2D, float] = {
    "x": ground_truth_df["x"][0],
    "y": ground_truth_df["y"][0],
}

In [ ]:
from sub import pdr, utils

importlib.reload(utils)

angle_df, cumulative_displacement_df = pdr.estimate_trajectory(
    acc_df,
    gyro_df,
    ground_truth_first_point=true_point,
)

utils.plot_displacement_map(
    map_dict,
    floor_name,
    dx,
    dy,
    cumulative_displacement_df,
    fig_size=fig_size,
)

In [ ]:
import numpy as np
import pandas as pd

# 軌跡前半の歩行軌跡の座標と強いBLEビーコンの位置座標の距離が最小になる角度を探索
# これは軌跡前半はドリフトが乗りづらいため
# 時間全体の中央を変数に入れる
center_of_time = ble_scans_df.tail(1)["ts"].to_numpy() / 2

first_half_strong_blescans = estimate.filter_strong_blescans(
    ble_scans_df,
    80,
    -73,
)
merged_strong_blescans_first_half = estimate.load_beacons_and_merge(
    BEACON_LIST_PATH,
    first_half_strong_blescans,
)


# マップ上にビーコンの位置をプロット
estimate.plot_map(map_dict, floor_name, dx, dy)

plt.colorbar(
    plt.scatter(
        merged_strong_blescans_first_half.x,
        merged_strong_blescans_first_half.y,
        c=merged_strong_blescans_first_half["ts"],
        cmap="rainbow",
    ),
)

plt.xlabel("x(m)")
plt.ylabel("y(m)")
plt.title("Beacons")
plt.show()

In [ ]:
rotated_cumulative_displacement_df = estimate.rotate_cumulative_displacement(
    cumulative_displacement_df,
    1.19,
    {"x": ground_truth_df.x[0], "y": ground_truth_df.y[0]},
)


# 同じ bdaddress, x, y を持つデータをグループ化
merged_df = pd.merge_asof(
    merged_strong_blescans_first_half,
    rotated_cumulative_displacement_df,
    on="ts",
    direction="nearest",
)

print(merged_df)


# マップ上にx_displacement,y_displacementとビーコンの位置をプロット
estimate.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        merged_df.x_displacement,
        merged_df.y_displacement,
        c=merged_df["ts"],
        cmap="rainbow",
    ),
)

# plt.colorbar(
# plt.scatter(
# merged_df.x,
# merged_df.y,
# c=merged_df["ts"],
# cmap="rainbow",
# ),
# )

plt.scatter(merged_df.x, merged_df.y, c="blue")


plt.xlabel("x_displacement(m)")
plt.ylabel("y_displacement(m)")
plt.title("Displacement")
plt.show()

In [ ]:
from sub import drift

first_time_remove_drift_angle, first_time_remove_drift_angle_displacement = (
    drift.remove_drift_in_angle_df(
        acc_df,
        angle_df,
        ground_truth_df,
    )
)

In [ ]:
from sub import ble

importlib.reload(ble)

rotated_optimal_angle_df, rotated_optimal_displacement_df = (
    ble.rotate_trajectory_to_optimal_alignment_using_ble(
        acc_df,
        first_time_remove_drift_angle,
        ble_scans_df,
        ble_positions_df,
        ground_truth_first_point={
            "x": ground_truth_df.x[0],
            "y": ground_truth_df.y[0],
        },
    )
)


estimate.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        rotated_optimal_displacement_df.x_displacement,
        rotated_optimal_displacement_df.y_displacement,
        c=rotated_optimal_displacement_df.ts,
        cmap="rainbow",
        s=5,
    ),
)
plt.xlabel("x_displacement(m)")
plt.ylabel("y_displacement(m)")
plt.title("Displacement")
plt.show()

In [ ]:
print(rotated_optimal_angle_df)

optimal_drift_and_euclidean = estimate.search_optimal_drift_from_angle(
    rotated_optimal_angle_df,
    ground_truth_df,
)


first_time_remove_drift_angle = pd.DataFrame(
    {
        "ts": rotated_optimal_angle_df.ts,
        "x": rotated_optimal_angle_df.x
        - optimal_drift_and_euclidean["drift"]
        * (rotated_optimal_angle_df.ts - rotated_optimal_angle_df.ts.iloc[0]),
    },
)

# first_time_remove_drift_angle= pd.DataFrame({
#     "ts":rotated_optimal_angle_df.ts,
#     "x":rotated_optimal_angle_df.x-0.002999999999999989*(rotated_optimal_angle_df.ts-rotated_optimal_angle_df.ts.iloc[0])
# })


first_time_remove_drift_angle_displacement = estimate.calculate_cumulative_displacement(
    first_time_remove_drift_angle.ts,
    first_time_remove_drift_angle["x"],
    0.5,
    {"x": ground_truth_df.x[0], "y": ground_truth_df.y[0]},
    ground_truth_df["ts"][0],
)

estimate.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        first_time_remove_drift_angle_displacement.x_displacement,
        first_time_remove_drift_angle_displacement.y_displacement,
        c=first_time_remove_drift_angle_displacement.ts,
        cmap="rainbow",
        s=5,
    ),
)
plt.xlabel("x_displacement(m)")
plt.ylabel("y_displacement(m)")
plt.title("Displacement")
plt.show()

In [ ]:
# 軌跡全体の歩行軌跡の座標と強いBLEビーコンの位置座標の距離が最小になる角度を探索見つける
overall_strong_blescans = estimate.filter_strong_blescans(
    ble_scans_df,
    ble_scans_df.tail(1)["ts"].values[0],
    -76,
)
merged_strong_blescans_overall = estimate.load_beacons_and_merge(
    BEACON_LIST_PATH,
    overall_strong_blescans,
)

overall_strong_ble_angle = estimate.search_optimal_angle(
    first_time_remove_drift_angle_displacement,
    ground_truth_df,
    merged_strong_blescans_overall,
)

rotate_by_overall_strong_ble_angle = pd.DataFrame(
    {
        "ts": first_time_remove_drift_angle.ts,
        "x": first_time_remove_drift_angle.x + overall_strong_ble_angle,
    },
)


rotate_by_overall_strong_ble_angle_displacement = (
    estimate.calculate_cumulative_displacement(
        rotate_by_overall_strong_ble_angle.ts,
        rotate_by_overall_strong_ble_angle["x"],
        0.5,
        {"x": ground_truth_df.x[0], "y": ground_truth_df.y[0]},
        ground_truth_df["%time"][0],
    )
)

estimate.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        rotate_by_overall_strong_ble_angle_displacement.x_displacement,
        rotate_by_overall_strong_ble_angle_displacement.y_displacement,
        c=rotate_by_overall_strong_ble_angle_displacement.ts,
        cmap="rainbow",
        s=5,
    ),
)
plt.xlabel("x_displacement(m)")
plt.ylabel("y_displacement(m)")
plt.title("Displacement")
plt.show()

In [ ]:
second_optimal_drift_and_euclidean = estimate.search_optimal_drift_from_angle(
    rotate_by_overall_strong_ble_angle,
    ground_truth_df,
)

second_time_remove_drift_angle = pd.DataFrame(
    {
        "ts": rotate_by_overall_strong_ble_angle.ts,
        "x": rotate_by_overall_strong_ble_angle.x
        - second_optimal_drift_and_euclidean["drift"]
        * (
            rotate_by_overall_strong_ble_angle.ts
            - rotate_by_overall_strong_ble_angle.ts.iloc[0]
        ),
    },
)

second_time_remove_drift_angle_displacement = (
    estimate.calculate_cumulative_displacement(
        second_time_remove_drift_angle.ts,
        second_time_remove_drift_angle["x"],
        0.5,
        {"x": ground_truth_df.x[0], "y": ground_truth_df.y[0]},
        ground_truth_df["%time"][0],
    )
)

estimate.plot_map(map_dict, floor_name, dx, dy)
plt.colorbar(
    plt.scatter(
        second_time_remove_drift_angle_displacement.x_displacement,
        second_time_remove_drift_angle_displacement.y_displacement,
        c=second_time_remove_drift_angle_displacement.ts,
        cmap="rainbow",
        s=5,
    ),
)
plt.xlabel("x_displacement(m)")
plt.ylabel("y_displacement(m)")
plt.title("Displacement")
plt.show()

In [ ]:
np.set_printoptions(threshold=np.inf)

correct_unpassable_displacement = estimate.correct_unpassable_points(
    second_time_remove_drift_angle_displacement[
        second_time_remove_drift_angle_displacement["ts"] < 180
    ],
    map_dict,
    floor_name,
    dx,
    dy,
)

estimate.plot_map(
    map_dict,
    floor_name,
    dx,
    dy,
)

plt.colorbar(
    plt.scatter(
        correct_unpassable_displacement["x_displacement"],
        correct_unpassable_displacement["y_displacement"],
        c=correct_unpassable_displacement["ts"],
        cmap="rainbow",
        s=5,
    ),
)


plt.xlabel("x(m)")
plt.ylabel("y(m)")
plt.title("correct_unpassable_displacement")
plt.show()


output_df = pd.DataFrame(
    {
        "ts": correct_unpassable_displacement["ts"],
        "x": correct_unpassable_displacement["x_displacement"],
        "y": correct_unpassable_displacement["y_displacement"],
        "floor": floor_name,
    },
)